In [5]:
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
import os

In [6]:
api_grok = "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"
api_langchain = "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"

In [7]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = api_langchain

In [8]:
if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = api_grok

from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="mixtral-8x7b-32768",
    temperature=0.7,
    max_tokens=2048
)

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={'device': 'cpu'}
)

In [12]:
# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

In [13]:
# Initialize conversation memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)


/tmp/ipykernel_51856/3916923272.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [14]:
def load_pdfs(pdf_path, is_directory=False):
    if is_directory:
        loader = DirectoryLoader(
            pdf_path,
            glob="**/*.pdf",
            loader_cls=PyPDFLoader
        )
    else:
        loader = PyPDFLoader(pdf_path)
    
    return loader.load()

In [15]:
def create_vectorstore(documents):
    # Split documents into chunks
    texts = text_splitter.split_documents(documents)
    
    # Create vector store
    vectorstore = Chroma.from_documents(
        documents=texts,
        embedding=embeddings
    )
    
    return vectorstore

In [16]:
# Create the conversational chain
def setup_chain(vectorstore):
    """
    Set up the conversational chain
    
    Args:
        vectorstore (Chroma): Vector store
        
    Returns:
        ConversationalRetrievalChain: Conversational chain
    """
    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(
            search_kwargs={"k": 3}
        ),
        memory=memory,
        return_source_documents=True
    )
    
    return chain

In [17]:
# Query function
def query_documents(chain, question):
    """
    Query the RAG system
    
    Args:
        chain (ConversationalRetrievalChain): Conversational chain
        question (str): User question
        
    Returns:
        dict: Response containing answer and source documents
    """
    response = chain({"question": question})
    return {
        "answer": response["answer"],
        "source_documents": response["source_documents"]
    }


In [18]:
if __name__ == "__main__":
    # Load documents
    pdf_path = "/home/updog/ragllm/downloaded_pdfs/تربية الأغنام : سلالة تمحضيت.pdf"  
    documents = load_pdfs(pdf_path, is_directory=False)
    
    # Create vector store
    vectorstore = create_vectorstore(documents)
    
    # Setup chain
    chain = setup_chain(vectorstore)
    
    # Query example
    question = "What is the main topic of the documents?"
    response = query_documents(chain, question)
    
    # Print results
    print("Answer:", response["answer"])
    print("\nSources:")
    for doc in response["source_documents"]:
        print(f"- {doc.page_content[:200]}...")

/tmp/ipykernel_51856/1132816632.py:13: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain({"question": question})


ValueError: Got multiple output keys: dict_keys(['answer', 'source_documents']), cannot determine which to store in memory. Please set the 'output_key' explicitly.